<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/amazon_product_selection_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [ ]:
# Create a table to count
product_count = df.groupby("product_id").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "reviews_count")
product_count.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B00FAADBHO|         1322|
|B000E13JGM|            2|
|B00WELAQ02|          342|
|B00JAIEAHC|          182|
|B009H0XQQY|          481|
|B0080E4040|           15|
|B00CJ4KM7G|           27|
|B00ZZOXCOC|           88|
|B00AHUQ15Q|            3|
|B014O1H0G2|            1|
|B000Y9YJI6|           67|
|B00LGZ8ELK|          367|
|B00NIG89DS|          169|
|B005LS2J14|          418|
|B00HRM8CUY|           86|
|B008BVKVGS|          100|
|B00JL4CIWE|           11|
|B00RH6A3N4|           34|
|B00PC58ADC|           25|
|B00J3A9L6W|           20|
+----------+-------------+
only showing top 20 rows



In [ ]:
products_filtered = product_count.filter(product_count.reviews_count > 2000)
products_filtered.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B001F42MKG|         2117|
|B00JG8H09Q|         3139|
|B0040IUI46|         3795|
|B0096M8VR2|         2336|
|B00F0DD0I6|         3047|
|B007PRHNHO|         2567|
|B00DU2CHE2|         2854|
|B00DYQYLQQ|         2119|
|B002TLTGM6|         2453|
|B004WYA852|         2161|
|B009UZ3TF4|         3171|
|B00BWF5U0M|         5356|
|B003VNKNEQ|         4218|
|B005AOKW8Q|         7098|
|B005890G8Y|         4468|
|B000J01I1G|         2048|
|B005E7KMH0|         2301|
|B008GFRE5A|         6823|
|B0057Z7EIY|         5522|
|B0051QVESA|         5811|
+----------+-------------+
only showing top 20 rows

